In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("data/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Lebu,-37.6167,-73.6500,47.28,87,4,5.10,CL,2024-05-11 01:59:42
1,1,Port-Aux-Francais,-49.3500,70.2167,41.22,71,7,31.12,TF,2024-05-11 01:59:42
2,2,Vallenar,-28.5708,-70.7581,53.58,75,100,2.73,CL,2024-05-11 01:59:42
3,3,Puerto Natales,-51.7236,-72.4875,29.44,92,100,3.24,CL,2024-05-11 01:59:42
4,4,Bodo,67.2800,14.4050,39.24,89,0,12.66,NO,2024-05-11 01:59:42


In [6]:
scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    hover_cols=['City'],
    width=700,
    height=500
)

# Show the plot
scatter_plot

:Scatter   [Lng]   (Lat,Humidity,City)